<a href="https://colab.research.google.com/github/jdz014/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/LS_DS12_232_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

Your code starts here:

In [121]:
!wget 'https://raw.githubusercontent.com/washingtonpost/data-school-shootings/master/school-shootings-data.csv'

--2020-02-26 02:29:43--  https://raw.githubusercontent.com/washingtonpost/data-school-shootings/master/school-shootings-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70530 (69K) [text/plain]
Saving to: ‘school-shootings-data.csv.4’

school-shootings-da 100%[===================>]  68.88K  --.-KB/s    in 0.02s   

2020-02-26 02:29:43 (2.77 MB/s) - ‘school-shootings-data.csv.4’ saved [70530/70530]



In [122]:
import pandas as pd

df = pd.read_csv('school-shootings-data.csv')
print(df.shape)
df.head()

(238, 50)


,uid,nces_school_id,school_name,nces_district_id,district_name,date,school_year,year,time,day_of_week,city,state,school_type,enrollment,killed,injured,casualties,shooting_type,age_shooter1,gender_shooter1,race_ethnicity_shooter1,shooter_relationship1,shooter_deceased1,deceased_notes1,age_shooter2,gender_shooter2,race_ethnicity_shooter2,shooter_relationship2,shooter_deceased2,deceased_notes2,white,black,hispanic,asian,american_indian_alaska_native,hawaiian_native_pacific_islander,two_or_more,resource_officer,weapon,weapon_source,lat,long,staffing,low_grade,high_grade,lunch,county,state_fips,county_fips,ulocale
0,1,080480000707,Columbine High School,804800.0,Jefferson County R-1,4/20/1999,1998-1999,1999,11:19 AM,Tuesday,Littleton,Colorado,public,1965,13,21,34,indiscriminate,18.0,m,w,student,1.0,suicide,17.0,m,w,student,1.0,suicide,1783,16.0,112.0,42.0,12.0,NaN,NaN,1,12-gauge Savage-Springfield 67H pump-action sh...,purchased from friends,39.603910,-105.075000,89.6,9,12,41.0,Jefferson County,8,8059,21.0
1,2,220054000422,Scotlandville Middle School,2200540.0,East Baton Rouge Parish School Board,4/22/1999,1998-1999,1999,12:30 PM,Thursday,Baton Rouge,Louisiana,public,588,0,1,1,targeted,14.0,m,NaN,former student (expelled),0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,583.0,0.0,0.0,0.0,NaN,NaN,0,.22-caliber handgun,NaN,30.529958,-91.169966,39.0,6,8,495.0,East Baton Rouge Parish,22,22033,12.0
2,3,130441001591,Heritage High School,1304410.0,Rockdale County,5/20/1999,1998-1999,1999,8:03 AM,Thursday,Conyers,Georgia,public,"1,369",0,6,6,indiscriminate,15.0,m,w,student,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1189,136.0,28.0,15.0,1.0,NaN,NaN,1,".22-caliber rifle, .357-caliber Magnum handgun",NaN,33.626922,-84.047960,84.0,9,12,125.0,Rockdale County,13,13247,21.0
3,4,421899003847,John Bartram High School,4218990.0,Philadelphia City SD,10/4/1999,1999-2000,1999,10:00 AM,Monday,Philadelphia,Pennsylvania,public,3147,0,1,1,targeted,17.0,m,NaN,student,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209,2736.0,27.0,170.0,5.0,NaN,NaN,1,RG .25-caliber handgun,purchased from friend,39.921509,-75.234108,41.0,9,12,2007.0,Philadelphia County,42,42101,11.0
4,5,250279000225,Dorchester High School,2502790.0,Boston,11/3/1999,1999-2000,1999,7:40 AM,Wednesday,Boston,Massachusetts,public,"1,116",0,1,1,targeted,NaN,m,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,755.0,287.0,29.0,5.0,NaN,NaN,0,NaN,NaN,42.285268,-71.075901,NaN,9,12,543.0,Suffolk County,25,25025,11.0


In [0]:
# Replace shooting type with 'other' for rows not 'targeted' or 'indiscriminate'
  df['shooting_type'] = df['shooting_type'].replace(['accidental', 'unclear',
                                                    'targeted and indiscriminate',
                                                    'public suicide',
                                                    'hostage suicide',
                                                    'accidental or targeted',
                                                    'public suicide (attempted)'],
                                                    'other')

# Fill missing value with 'other'
  df['shooting_type'] = df['shooting_type'].fillna('other')

In [124]:
# Majority class baseline 59%
df['shooting_type'].value_counts(normalize=True)

targeted          0.588235
other             0.222689
indiscriminate    0.189076
Name: shooting_type, dtype: float64

In [125]:
from sklearn.model_selection import train_test_split

# Create train, test
train, test = train_test_split(df, train_size=0.80, random_state=21, stratify=df['shooting_type'])

train.shape, test.shape

((190, 50), (48, 50))

In [0]:
def wrangle(df):

  # Avoid SettingWithCopyWarning
  df = df.copy()

  # Remove commas from numbers
  df['white'] = df['white'].str.replace(",", "")

  # Change from object to int
  df['white'] = pd.to_numeric(df['white'])
  
  # Remove commas from numbers
  df['enrollment'] = df['enrollment'].str.replace(",", "")

  # Change from object to int
  df['enrollment'] = pd.to_numeric(df['enrollment'])

  # Fill missing values for these specific columns
  df.fillna({'white': 0, 'black': 0, 'hispanic': 0, 'asian': 0,
           'american_indian_alaska_native': 0,
           'hawaiian_native_pacific_islander': 0, 'two_or_more': 0,
           'district_name': 'Unknown', 'time': '12:00 PM', 'lat': 33.612910,
           'long': -86.682000, 'staffing': 60.42, 'low_grade': '9',
           'high_grade': '12'}, inplace=True)
  
  # Drop columns with 200+ missing values
  df = df.drop(columns=['deceased_notes1', 'age_shooter2', 'gender_shooter2', 
                        'race_ethnicity_shooter2', 'shooter_relationship2', 
                        'shooter_deceased2', 'deceased_notes2'])

  # Drop unusable variance 
  df = df.drop(columns=['uid', 'nces_school_id', 'nces_district_id', 'weapon', 
                        'weapon_source', 'state_fips', 'county_fips', 'ulocale',
                        'lunch', 'age_shooter1', 'gender_shooter1',
                        'race_ethnicity_shooter1', 'shooter_relationship1',
                        'shooter_deceased1'])
  
  # Change date to datettime
  df['date'] = pd.to_datetime(df['date'])

  return df

train = wrangle(train)
test = wrangle(test)

In [127]:
train.shape, test.shape

((190, 29), (48, 29))

In [128]:
!pip install category_encoders==2.*

In [164]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.linear_model import Ridge

target = 'shooting_type'
features = train.columns.drop([target, 'date'])
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    StandardScaler(), 
    RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=21)
)

k = 20
scores = cross_val_score(pipeline, X_train, y_train, cv=k)
print(f'MAE for {k} folds:', scores)

MAE for 20 folds: [0.4        0.5        0.5        0.8        0.6        0.7
 0.5        0.8        0.7        0.4        0.66666667 0.44444444
 0.55555556 0.44444444 0.44444444 0.55555556 0.66666667 0.55555556
 0.44444444 0.44444444]


In [165]:
scores.mean()

0.5561111111111111

In [155]:
from sklearn.tree import DecisionTreeClassifier

target = 'shooting_type'
features = train.columns.drop([target, 'date', ])
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    DecisionTreeClassifier(max_depth=3)
)

pipeline.fit(X_train, y_train)
print('Test Accuracy:', pipeline.score(X_test, y_test))

Test Accuracy: 0.5416666666666666
